In [1]:
from __future__ import print_function

import SimpleITK as sitk
from ipywidgets import interact, fixed
import itk
from itkwidgets import view

import numpy as np
import pandas as pd

import os, re, sys
import gui
%matplotlib notebook

#### Setting affine registration parameter

In [ ]:
def get_affine_paramater() :
    
    params = sitk.GetDefaultParameterMap('affine')
    params['FixedInternalImagePixelType'] = ['short']
    params['MovingInternalImagePixelType'] = ['short']
    params['FixedImageDimension'] = ['3']
    params['MovingImageDimension'] = ['3']
    params['UseDirectionCosines'] = ['true']

    ################## Main Components #############################
    params['Registration'] = ['MultiResolutionRegistration']
    params['Interpolator'] =['BSplineInterpolator']
    params['ResampleInterpolator'] = ['FinalBSplineInterpolator']
    params['Resampler'] = ['DefaultResampler']

    params['FixedImagePyramid'] = ['FixedRecursiveImagePyramid']
    params['MovingImagePyramid'] = ['MovingRecursiveImagePyramid']

    params['Optimizer'] = ['AdaptiveStochasticGradientDescent']
    params['Transform'] = ['EulerTransform']
    params['Metric'] = ['AdvancedMattesMutualInformation']

    #################### Transformation ##########################

    params['AutomaticScalesEstimation'] = ['true']
    params['AutomaticTransformInitialization'] = ['true']
    params['HowToCombineTransforms'] = ['Compose']

    ################### Similarity measure #####################

    params['NumberOfHistogramBins'] = ['32']
    params['ErodeMask'] = ['false']

    #################### Multiresolution ######################

    params['NumberOfResolutions'] = ['3']
    params['ImagePyramidSchedule'] = ['4', '4', '4', '2', '2', '2', '1', '1', '1']

    ################### Optimizer ############################

    params['MaximumNumberOfIterations'] = ['250']

    # The step size of the optimizer, in mm. By default the voxel size is used.
    # which usually works well. In case of unusual high-resolution images
    # (eg histology) it is necessary to increase this value a bit, to the size
    # of the "smallest visible structure" in the image:
    # (MaximumStepLength 1.0)

    ################ Image sampling ######################

    params['NumberOfSpatialSamples'] = ['10000']
    params['NewSamplesEveryIteration'] = ['true']
    params['ImageSampler'] = ['Random']

    ############# Interpolation and Resampling ##############

    params['BSplineInterpolationOrder'] =['1']
    params['FinalBSplineInterpolationOrder'] = ['1']
    params['DefaultPixelValue'] =['0']
    params['WriteResultImage'] = ['true']
    params['ResultImagePixelType'] = ['short']
    params['ResultImageFormat'] = ['mha']
    params['CompressResultImage'] =['false']
   
    return params

#### Setting bspline registration parameters

In [ ]:
def get_bspline_paramter() :
    
    params = sitk.GetDefaultParameterMap('bspline')
    params['FixedInternalImagePixelType'] = ['short']
    params['MovingInternalImagePixelType'] = ['short']
    params['FixedImageDimension'] = ['3']
    params['MovingImageDimension'] = ['3']
    params['UseDirectionCosines'] = ['true']

    ################## Main Components #############################
    params['Registration'] = ['MultiResolutionRegistration']
    params['Interpolator'] =['BSplineInterpolator']
    params['ResampleInterpolator'] = ['FinalBSplineInterpolator']
    params['Resampler'] = ['DefaultResampler']

    params['FixedImagePyramid'] = ['FixedRecursiveImagePyramid']
    params['MovingImagePyramid'] = ['MovingRecursiveImagePyramid']

    params['Optimizer'] = ['AdaptiveStochasticGradientDescent']
    params['Transform'] = ['BSplineTransform']
    params['Metric'] = ['AdvancedMattesMutualInformation']

    #################### Transformation ############################
    params['FinalGridSpacingInPhysicalUnits'] =['40']
    params['GridSpacingSchedule'] = ['4.0', '2.0', '1.0']
    params['HowToCombineTransforms'] = ['Compose']

    #################### Similarity measure ########################
    params['NumberOfHistogramBins'] =['32']
    params['ErodeMask'] = ['false']

    #################### Multiresolution ######################

    params['NumberOfResolutions'] = ['3']
    params['ImagePyramidSchedule'] = ['4', '4', '4', '2', '2', '2', '1', '1', '1']

    ################### Optimizer ############################

    params['MaximumNumberOfIterations'] = ['500']

    ################ Image sampling ######################

    params['NumberOfSpatialSamples'] = ['10000']
    params['NewSamplesEveryIteration'] = ['true']
    params['ImageSampler'] = ['Random']

    ############# Interpolation and Resampling ##############

    params['BSplineInterpolationOrder'] =['3']
    params['FinalBSplineInterpolationOrder'] = ['3']
    params['DefaultPixelValue'] =['0']
    params['WriteResultImage'] = ['true']
    params['ResultImagePixelType'] = ['short']
    params['ResultImageFormat'] = ['mha']
    params['CompressResultImage'] =['false']
    
    return params

In [ ]:
def command_iteration(filter) :
    print("{0:3} = {1:10.5f}".format(filter.GetElapsedIterations(),
                                     filter.GetMetric()))

In [ ]:
def get_outputpoints_pointset(search_str = "OutputIndexFixed") :
    '''
    This function return the pointset of one type of five coordinates
    such as "InputIndex", "InputPoint", "OutputIndexFixed", "OutputPoint", 
    and "OutputIndexMoving" 

    Input
        search_str:  
            - Type: string
            - Contents: one of five coordinates, default = OutputIndexFixed  
    Output
        pointset:
            - Type: numpy array 2D
            - Contents: pointset found with search_str 
    '''
    pointsets = []
    want_str = search_str + " = [ [0-9-\.]+ [0-9-\.]+ [0-9-\.]+ ]"
    p = re.compile(want_str) 
    ifp = open("outputpoints.txt", "r")
    line = ifp.readline()
    while line :
        splitted = line.split(";")
        for block in splitted :
            if p.match(block.strip()) :
                gotcha = re.search(r'= \[ ([0-9-\.]+ [0-9-\.]+ [0-9-\.]+) \]', 
                                   block).group(1)
                pointsets.append(np.array([float(x) for x in gotcha.split()]))
        line = ifp.readline()
    return np.array(pointsets)

In [ ]:
def put_fiji_landmarkfile(arr, outfilename="fiji_landmark.points") :
    '''
    This function write out the input file of landmark points of FIJI plugin

    Input
        arr:   
            - Type: numpy array 2D
            - Contents: one of five coordinates, default = OutputIndexFixed  
        outfilename:
            - Type: string
            - Contents: pointset outfilename
    
    '''
    
    if arr.ndim != 2 :
        print("arr shape should be (n, 3)")
        return np.nan
    ofp = open(outfilename, "w")
    preface = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE namedpointset [
  <!ELEMENT namedpointset (pointworld*)>
  <!ELEMENT pointworld EMPTY>
  <!ATTLIST namedpointset version CDATA #REQUIRED>
  <!ATTLIST pointworld set (true|false) #REQUIRED>
  <!ATTLIST pointworld name CDATA #REQUIRED>
  <!ATTLIST pointworld x CDATA #IMPLIED>
  <!ATTLIST pointworld y CDATA #IMPLIED>
  <!ATTLIST pointworld z CDATA #IMPLIED>
]>

<namedpointset version="1.0">"""
    ofp.write(preface + '\n')
    for i, x in enumerate(arr) :
        print(x)
        coordline = '  <pointworld set="true" name="Named Point ({})" x="{}" y="{}" z="{}"/>'\
                    .format(i, int(x[0]), int(x[1]), int(x[2]))
        ofp.write(coordline+"\n")
    ofp.write('</namedpointset>')   

In [ ]:
def marking_circles(img, pointset, circle_size = [3, 3, 1], 
                    circle_pixelvalue = 1000, pixelType = sitk.sitkFloat32) :
    '''
    This function prints circles with the coordinates of 
    the input point as the center on the input image.

    Input
        img: 
            - Type: SimpleITK.SimpleITK.Image
            - Contents: An image that circles will put on  
        pointset:
            - Type: numpy array 2d
            - Contents: circle's center coordinate
    Output
        img:
            - SimpleITK.SimpleITK.Image
            - Contents: an image that circles put on 
    '''
    pixeltype = pixelType
    img = sitk.Cast(img, pixeltype)
    for pnt in pointset :
        pnt_size = circle_size
        pnt_center = pnt.tolist()
        point_circle = sitk.GaussianSource(pixeltype, img.GetSize(), pnt_size, 
                                           pnt_center, circle_pixelvalue)
        point_circle.SetOrigin(img.GetOrigin())
        point_circle.SetSpacing(img.GetSpacing())
        point_circle.SetDirection(img.GetDirection())
        img = img - point_circle 
    return img

### Elastix : T(fixed_img) = moving_img
#### Our case
- fixed_img = supine
- moving_img = prone

In [ ]:
# read moving image and fixed image
moving_img_filename = "../preprocessings/03_51391773_prone.mha"
fixed_img_filename = "../preprocessings/03_51391773_supine.mha"

fixed_img = sitk.ReadImage(fixed_img_filename)
moving_img = sitk.ReadImage(moving_img_filename)

# SimpleITK registration supports only the pixel types sitkFloat32 and sitkFloat64
fixed_img = sitk.Cast(fixed_img, sitk.sitkFloat32)
moving_img = sitk.Cast(moving_img, sitk.sitkFloat32)

# rescale intensity for normalization
fixed_img = sitk.RescaleIntensity(fixed_img)
moving_img = sitk.RescaleIntensity(moving_img)

In [ ]:
gui.MultiImageDisplay(image_list = [moving_img, fixed_img],                   
                      title_list = ['Prone => moving', 'Supine => fixed'], 
                     figure_size=(8.4, 4.4))
#                       figure_size=(8,4), 
#                       window_level_list=[ct_window_level, mr_window_level]);

#### Doing affine registration

In [ ]:
elastixImageFilter = sitk.ElastixImageFilter()

elastixImageFilter.SetFixedImage(fixed_img)
elastixImageFilter.SetMovingImage(moving_img)

elastixImageFilter.SetParameterMap(get_affine_paramater())
elastixImageFilter.PrintParameterMap()
elastixImageFilter.Execute()

sitk.WriteImage(elastixImageFilter.GetResultImage(), "03_51391773a.mha")

In [ ]:
gui.MultiImageDisplay(image_list = [moving_img, sitk.ReadImage("03_51391773a.mha"), fixed_img],                   
                      title_list = ['Prone => moving', 'f2m_a', 'Supine => fixed'], 
                     figure_size=(10.4, 4.4))

#### Affine Deformationmap

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.LogToConsoleOn()
transformixImageFilter.ComputeDeformationFieldOn()
transformixImageFilter.SetMovingImage(moving_img)
transformixImageFilter.Execute()
affine_displacementField = transformixImageFilter.GetDeformationField()
sitk.WriteImage(affine_displacementField, "03_51391773_affine_displacementField.mha")

#### Bspline registration

In [ ]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixed_img)
elastixImageFilter.SetMovingImage(sitk.ReadImage("03_51391773a.mha"))

elastixImageFilter.SetParameterMap(get_bspline_paramter())
elastixImageFilter.PrintParameterMap()

elastixImageFilter.Execute()
sitk.WriteImage(elastixImageFilter.GetResultImage(), "03_51391773ab.mha")

In [ ]:
gui.MultiImageDisplay(image_list = [sitk.ReadImage("03_51391773a.mha"),
                                    elastixImageFilter.GetResultImage(), fixed_img],                   
                      title_list = ['affined_Prone => moving', 'f2m_ab', 'Supine => fixed'], 
                     figure_size=(10.4, 4.4))

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.LogToConsoleOn()
transformixImageFilter.ComputeDeformationFieldOn()
transformixImageFilter.SetMovingImage(sitk.ReadImage("03_51391773a.mha"))
transformixImageFilter.Execute()
bspline_displacementField = transformixImageFilter.GetDeformationField()
sitk.WriteImage(bspline_displacementField, "03_51391773_bspline_displacementField.mha")

#### Doing demon registration

In [ ]:
# moving_img = sitk.ReadImage("03_51391773ab.mha", sitk.sitkFloat32)
# fixed_img = sitk.ReadImage("../preprocessings/03_51391773_supine.mha", sitk.sitkFloat32)
# print(fixed_img.GetSize(), moving_img.GetSize())
# matcher = sitk.HistogramMatchingImageFilter()
# matcher.SetNumberOfHistogramLevels(1024)
# matcher.SetNumberOfMatchPoints(7)
# matcher.ThresholdAtMeanIntensityOn()
# moving_img = matcher.Execute(moving_img,fixed_img)

# demons = sitk.DiffeomorphicDemonsRegistrationFilter()
# demons.SetNumberOfIterations(10)
# # Standard deviation for Gaussian smoothing of displacement field
# demons.SetStandardDeviations(1.0)
# demons.AddCommand(sitk.sitkIterationEvent,lambda: command_iteration(demons) )
# diffeomorphic_displacementField = demons.Execute(fixed_img, moving_img)

# sitk.WriteImage(diffeomorphic_displacementField, "03_51391773_diffeomorphic_displacementField.mha")
# print("-------")
# print("Number Of Iterations: {0}".format(demons.GetElapsedIterations()))
# print(" RMS: {0}".format(demons.GetRMSChange()))
# #print(" RMS: {0}".format(demons.))
# #print("Metric: {}".format(demons.GetMetric()))
# diffeomorphicTx = sitk.DisplacementFieldTransform(diffeomorphic_displacementField)
# sitk.WriteTransform(diffeomorphicTx, 'demon1.txt')



# if (not"SITK_NOSHOW" in os.environ ):
#     resampler = sitk.ResampleImageFilter()
#     resampler.SetReferenceImage(fixed_img)
#     resampler.SetInterpolator(sitk.sitkLinear)
#     resampler.SetDefaultPixelValue(100)
#     resampler.SetTransform(outTx)
#     out = resampler.Execute(moving_img)
#     simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_img), sitk.sitkUInt8)
#     simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    
#     zeros = sitk.Image(simg1.GetSize(), simg1.GetPixelID())
#     zeros.CopyInformation(simg1)
#     # Use the // floor division operator so that the pixel type is
#     # the same for all three images which is the expectation for
#     # the compose filter.
#     #cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)
#     cimg = sitk.Compose(simg1, zeros, simg2)
#     sitk.Show(cimg, "DiffeomorphicDemonsRegistration")

In [ ]:
# resampler = sitk.ResampleImageFilter()
# resampler.SetReferenceImage(fixed_img)
# resampler.SetInterpolator(sitk.sitkLinear)
# resampler.SetDefaultPixelValue(100)
# resampler.SetTransform(diffeomorphicTx)
# out = resampler.Execute(moving_img)

# simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_img), sitk.sitkUInt8)
# simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
# zeros = sitk.Image(simg1.GetSize(), simg1.GetPixelID())
# zeros.CopyInformation(simg1)
# cimg = sitk.Compose(simg1, zeros, simg2)
# sitk.WriteImage(cimg, "03_51391773_reg_prone2supine_done_dfm.mha")

In [ ]:
# sitk.CompositeTransform([affine_displacementField, 
#                          bspline_displacementFiels, 
#                          diffeomorphic_displacementField])

In [ ]:
ifp = open("03_51391773_supine.txt", 'r')
line = ifp.readline()
line = ifp.readline()
line = ifp.readline()
supine_indices  =[]
while line :
    tmp = line.split()
    supine_indices.append([np.int64(x.split(".")[0]) for x in tmp])
    line = ifp.readline()
print(supine_indices)

for ind in supine_indices : 
    pnt = fixed_img.TransformIndexToPhysicalPoint((int(ind[0]), int(ind[1]), int(ind[2])))
    print(pnt)

In [ ]:
ifp = open("03_51391773_prone.txt", 'r')
line = ifp.readline()
line = ifp.readline()
line = ifp.readline()
prone_indices  =[]
while line :
    tmp = line.split()
    prone_indices.append([np.int64(x.split(".")[0]) for x in tmp])
    line = ifp.readline()
print(prone_indices)

In [ ]:
# Affine
affine_displacementField = sitk.Cast(sitk.ReadImage('../preprocessings/03_51391773_dls.mha'), 
                                       sitk.sitkVectorFloat64)
affinedTx = sitk.DisplacementFieldTransform(sitk.Cast(affine_displacementField, 
                                                      sitk.sitkVectorFloat64))
affine_displacement_image = affinedTx.GetDisplacementField()
affinedTx_inverse = sitk.DisplacementFieldTransform(
                         sitk.InvertDisplacementField(affine_displacement_image,
                                                    maximumNumberOfIterations = 20,
                                                    maxErrorToleranceThreshold = 0.01,
                                                    meanErrorToleranceThreshold = 0.0001,
                                                    enforceBoundaryCondition = True))

# Bspline
bsplinedTx = sitk.DisplacementFieldTransform(sitk.Cast(bspline_displacementField, 
                                                      sitk.sitkVectorFloat64))
bspline_displacement_image = bsplinedTx.GetDisplacementField()
bsplinedTx_inverse = sitk.DisplacementFieldTransform(
                         sitk.InvertDisplacementField(bspline_displacement_image,
                                                    maximumNumberOfIterations = 20,
                                                    maxErrorToleranceThreshold = 0.01,
                                                    meanErrorToleranceThreshold = 0.0001,
                                                    enforceBoundaryCondition = True))

# DiffeomorphicTx
# diffeomorphicTx = sitk.DisplacementFieldTransform(sitk.Cast(diffeomorphic_displacementField,
#                                                             sitk.sitkVectorFloat64))
# diffeomorphic_displacement_image = diffeomorphicTx.GetDisplacementField()
# diffeomorphicTx_inverse = sitk.DisplacementFieldTransform(
#                               sitk.InvertDisplacementField(diffeomorphic_displacement_image,
#                                                           maximumNumberOfIterations = 20,
#                                                           maxErrorToleranceThreshold = 0.01,
#                                                           meanErrorToleranceThreshold = 0.0001,
#                                                           enforceBoundaryCondition = True))


In [ ]:
levelset_displacementField = sitk.Cast(sitk.ReadImage('../preprocessings/03_51391773_dls.mha'), 
                                       sitk.sitkVectorFloat64)
levelsetTx = sitk.DisplacementFieldTransform(levelset_displacementField)
levelset_displacement_image = levelsetTx.GetDisplacementField()
levelsetTx_inverse = sitk.DisplacementFieldTransform(
                          sitk.InvertDisplacementField(levelset_displacement_image,
                                                      maximumNumberOfIterations = 20,
                                                      maxErrorToleranceThreshold = 0.01,
                                                      meanErrorToleranceThreshold = 0.0001,
                                                      enforceBoundaryCondition = True))

In [ ]:
reg_pnts = pd.DataFrame(columns=["x_p", "y_p", "z_p", "x_a", "y_a", "z_a", 
                                 "x_b", "y_b", "z_b", "x_d", "y_d", "z_d", 
                                 "x_s", "y_s", "z_s"])
for prone_ind, supine_ind in zip(prone_indices, supine_indices) : 
    pnt = moving_img.TransformIndexToPhysicalPoint(\
                    (int(prone_ind[0]), int(prone_ind[1]), int(prone_ind[2])))
    pnt_fixed1 = affinedTx_inverse.TransformPoint(pnt)
    pnt_fixed2 = bsplinedTx_inverse.TransformPoint(pnt_fixed1)
    #pnt_fixed3 = diffeomorphicTx_inverse.TransformPoint(pnt_fixed2)
    pnt_fixed3 = levelsetTx_inverse.TransformPoint(pnt_fixed2)
    print('affined: ', pnt_fixed1)
    print('bsplined: ', pnt_fixed2)
    print("levelset: ", pnt_fixed3)
    pnt_sup = fixed_img.TransformIndexToPhysicalPoint((int(supine_ind[0]), 
                                                             int(supine_ind[1]), 
                                                             int(supine_ind[2])))
    reg_pnts.loc[len(reg_pnts)] = [pnt[0], pnt[1], pnt[2], 
                                  pnt_fixed1[0], pnt_fixed1[1], pnt_fixed1[2], 
                                  pnt_fixed2[0], pnt_fixed2[1], pnt_fixed2[2], 
                                  pnt_fixed3[0], pnt_fixed3[1], pnt_fixed3[2], 
                                  pnt_sup[0], pnt_sup[1], pnt_sup[2]]
     #there_and_back = np.array(affinedTx_inverse.TransformPoint(affinedTx.TransformPoint(pnt)))
    #print(there_and_back)

In [ ]:
reg_pnts['TRE_aff'] = reg_pnts.apply(lambda x: np.sqrt((x['x_s']-x['x_a'])**2 + \
                                                       (x['y_s']-x['y_a'])**2 + \
                                                       (x['z_s']-x['z_a'])**2), axis = 1)

reg_pnts['TRE_bspl'] = reg_pnts.apply(lambda x: np.sqrt((x['x_s']-x['x_b'])**2 + \
                                                       (x['y_s']-x['y_b'])**2 + \
                                                       (x['z_s']-x['z_b'])**2), axis = 1)

reg_pnts['TRE_lvlset'] = reg_pnts.apply(lambda x: np.sqrt((x['x_s']-x['x_d'])**2 + \
                                                       (x['y_s']-x['y_d'])**2 + \
                                                       (x['z_s']-x['z_d'])**2), axis = 1)
reg_pnts.to_csv("03_51391773_registration_TRE.csv", index=True)

In [ ]:
#diffeomorphicTx = sitk.ReadTransform('demon1.txt')
#diffeomorphic_displacement_image = demonTx.GetDisplacementField()
diffeomorphic_displacement_image = sitk.ReadImage("03_51391773_diffeomorphic_displacementField.mha")
diffeomorphicTx_inverse = sitk.DisplacementFieldTransform(
                         sitk.InvertDisplacementField(diffeomorphic_displacement_image,
                                                    maximumNumberOfIterations = 20,
                                                    maxErrorToleranceThreshold = 0.01,
                                                    meanErrorToleranceThreshold = 0.0001,
                                                    enforceBoundaryCondition = True))

In [ ]:
# demonTx = sitk.DisplacementFieldTransform(sitk.Cast(diffeomorphic_displacementField, 
#                                                       sitk.sitkVectorFloat64))
# demon_displacement_image = demonTx.GetDisplacementField()
# demonTx_inverse = sitk.DisplacementFieldTransform(
#                          sitk.InvertDisplacementField(demon_displacement_image,
#                                                     maximumNumberOfIterations = 20,
#                                                     maxErrorToleranceThreshold = 0.01,
#                                                     meanErrorToleranceThreshold = 0.0001,
#                                                     enforceBoundaryCondition = True))


In [ ]:
pnt = [0.4, 1.2, 1.2]

affinedTx = sitk.DisplacementFieldTransform(sitk.Cast(affine_displacementField, 
                                                      sitk.sitkVectorFloat64))
secondary_transformed = np.array(affinedTx.TransformPoint(pnt))
print(secondary_transformed)

displacement_image = affinedTx.GetDisplacementField()
affinedTx_inverse = sitk.DisplacementFieldTransform(
                         sitk.InvertDisplacementField(displacement_image,
                                                    maximumNumberOfIterations = 20,
                                                    maxErrorToleranceThreshold = 0.01,
                                                    meanErrorToleranceThreshold = 0.0001,
                                                    enforceBoundaryCondition = True))
there_and_back = np.array(affinedTx_inverse.TransformPoint(affinedTx.TransformPoint(pnt)))
print(there_and_back)
#print(f'Deformaiton field transformation result: {secondary_transformed}')
#bsplineTx = sitk.DisplacementFieldTransform(sitk.Cast(bspline_displacementField, 
#                                                      sitk.sitkVectorFloat64))
#diffeomorphicTx = sitk.DisplacementFieldTransform(sitk.Cast(diffeomorphic_displacementField, 
#                                                           sitk.sitkVectorFloat64))

#sitk.TransformToDisplacementField()